# Despliegue del modelo

### Librerías

In [ ]:
# !pip install azureml-sdk
# !pip install pandas
# !pip instal scikit-learn
# !pip install ipykernel 

In [3]:
import json

In [6]:
# Leer archivo json con credenciales

id = open('credentials.json', 'r')
mi = json.load(id)

### Crear grupo de recursos y cargar modelo

In [7]:
# Crear grupo de recursos
from azureml.core import Workspace

ws = Workspace.create(name="wsmodel_in",
                      subscription_id = mi["id"],
                      resource_group = "rgmodel_in",
                      location = "centralindia")

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-33e08e9b8d
Deploying KeyVault with name wsmodelikeyvault34a9d312.
Deploying StorageAccount with name wsmodelistoragedee5974bb.
Deploying Workspace with name wsmodel_in.
Deploying AppInsights with name wsmodeliinsights9e89d411.
Deployed AppInsights with name wsmodeliinsights9e89d411. Took 39.57 seconds.
Deployed Workspace with name wsmodel_in. Took 42.21 seconds.


### Cargar modelo al grupo de recursos

In [8]:
# Cargar modelo a grupo de recursos
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="dates_model",
                                  workspace=ws)

Registering model dates_model


### Crear archivo score.py

In [9]:
scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('dates_model')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    result = model.predict(data)

    return json.dumps({"result": result.tolist()})
  except Exception as e:
    return json.dumps({"error": str(e)})
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

### Entorno virtual y dependencias

In [10]:
from azureml.core.environment import Environment
virtual_env = Environment("env_bank")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

### Configuración de inferencia y despliegue del modelo

In [11]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='bankrup-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Esteban\AppData\Local\Temp\ipykernel_37108\2900293506.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [12]:
# Deployment
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


### Guardar uri para utilizar el modelo

In [13]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [ ]:
print(scoring_uri)